# Intro to neural networks: Regression

This notebook is based on the SEG Geophysical Tutorial from August 2018 by Graham Ganssle: https://github.com/seg/tutorials-2018.

The idea is to introduce the based components of an artificial neural network and implement a simple version of one using Numpy.

We'll use a regression task — predicting a DT log from other logs.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

## Activation functions

A neural network is nothing but a nonlinear system of equations like $\mathbf{y} = \sigma(\mathbf{W}\mathbf{x} + \mathbf{b})$.

There are multiple functions $\sigma$ that are used to introduce the non-linear component. One of the earliest was the *sigmoid* (aka *logistic*) function is given by:

$$ \sigma(z) = \frac{1}{1 + \operatorname{e}^{-z}} $$

Its derivative is:

$$ \frac{\mathrm{d} \sigma(z)}{\mathrm{d} z} = \sigma(z)  (1 - \sigma(z)) $$

We need the derivative for the _backpropagation_ process that enables neural networks to learn efficiently. Backpropagation adjusts the parameters of the neural network by injecting an error signal backwards through the network's layers, from the last to the first.

We can implement the logistic function like this in Python:

In [ ]:
def logistic(z, derivative=False):
    if not derivative:
        return 1 / (1 + np.exp(-z))
    else:
        return z * (1 - z)  # In the implementation, 'z' will actually be sigma(z).

The function transforms, or 'squeezes', numbers into the range [0, 1] and looks like this:

In [ ]:
from utils import plot_activation

plot_activation(logistic)

In practice, while this function is sometimes useful for handling probabilities, there are some problems with it.

- The maximum value of the derivative is 0.25, which tends to reduce the learning rate, especially in deeper layers.
- Large activations input result in 'saturation' and a gradient of 0 ('vanishing gradient'), which will halt learning.
- The exponentials are expensive to compute.

The $\operatorname{tanh}$ function solves some of these issues — for example, it has a maximum gradient of 1.

In [ ]:
def tanh(z, derivative=False):
    """
    Compute a tanh transformation for a given input.
    """
    if not derivative:
        return (np.exp(z) - np.exp(-z)) / (np.exp(z) + np.exp(-z))
    else:
        return 1 - z**2  # In the implementation, we'll get tanh(z) coming at us.

In [ ]:
plot_activation(tanh)

But it still suffers from the saturation issue, and the expense of computation.

Both of these issues are solved by the ReLU, or rectified linear unit, function.

### EXERCISE

The **rectified linear unit** (ReLU) and its derivative are given by:

$$ f(z) = \begin{cases}
    z & \text{if } z > 0, \\
    0 & \text{otherwise}.
\end{cases} $$

$$ \frac{\mathrm{d}f(z)}{\mathrm{d}z} = \begin{cases}
    1 & \text{if } z > 0, \\
    0 & \text{otherwise}.
\end{cases} $$

The main problem with the ReLU is that, depending on how the weights are initialized, some units in the network might 'die' as they get into negative activations and never fire. Accordingly, a common variant of the ReLU is the 'parametric' ReLU, which has $f(z) = \alpha z$, when $Z \leq 0$ (the corresponding derivative is then just $\alpha$). The parameter $\alpha$ can be tuned like other hyperparameters. A typical value is 0.01.

The parametric ReLU is also called a 'leaky' ReLU, but that term implies that the value of $\alpha$ is not being considered as a hyperparameter or tuned in any way.

Can you implement a ReLU? (Or, if you prefer, a parametric ReLU?)

In [ ]:
# Note that if you use `if z > 0` in your code, then
# the plot_activation function won't work, because it
# defines z as an array to make its plots. In general,
# it's a good idea to write functions that work for
# both scalars and arrays, where possible.

# YOUR CODE HERE



In [ ]:
# ReLU solution.
def relu(z, derivative=True):
    """
    Compute a Rectified Linear Unit transformation for a given input.
    """
    if not derivative:
        return z * (z > 0)
    else:
        return 1 * (z > 0)    

In [ ]:
# ALTERNATIVE
# Parametric ReLU and a ReLU that builds off it.
def prelu(z, derivative=False, alpha=0.1):
    """
    Compute a Parameter Rectified Linear Unit transformation for a given input.
    """
    if not derivative:
        return np.maximum(alpha * z, z)  # Assumes alpha <= 1.
    else:
        return alpha * (z <= 0) + (z > 0)
    
def relu(z, derivative=False):
    """
    An ordinary ReLU.
    """
    return prelu(z, alpha=0, derivative=derivative)

In [ ]:
assert (relu(-1), relu(0), relu(1)) == (0, 0, 1)

In [1]:
plot_activation(relu)

NameError: name 'plot_activation' is not defined

### Stretch exercise

Some people prefer the exponential linear unit, because it has a smooth derivative. Can you implement it?

$$ f(z) = \begin{cases} z & \text{if } z > 0 \\  \alpha(e^z-1) & \text{otherwise} \end{cases} $$

The derivative is given by:

$$ \frac{\mathrm{d} f}{\mathrm{d} z} = \begin{cases} 1 & \text{if } z > 0 \\  \alpha e^z & \text{otherwise} \end{cases} $$

Again, $\alpha$ is a hyperparameter.

In [ ]:
# YOUR CODE HERE



Check the [Intro_to_neural_network_regression.ipynb](../master/Intro_to_neural_network_regression.ipynb) master notebook for a solution to this problem. 

In [ ]:
def elu(z, derivative=False, alpha=0.01):
    """
    Compute an Exponential Linear Unit transformation for a given input.
    """
    if not derivative:
        return z * (z > 0) + alpha * (np.exp(z) - 1) * (z <= 0)
    else:
        return 1 * (z > 0) + alpha * np.exp(z) * (z <= 0)

In [ ]:
plot_activation(elu, alpha=0.25)

There are still other rectifiers — e.g. the GELU and SiLU — read about them [on Wikipedia](https://en.wikipedia.org/wiki/Rectifier_(neural_networks)). Why not try implementing some of them?

## Loss

We're going to need a way to tell when we're doing well. The **loss function** is some measure of error. We'll use the mean squared error, where the error is the difference between a known value of the target and our estimate of the target.

We're going to need a function for that too:

In [ ]:
def loss(y, y_hat):
    """
    Compute half the mean squared error. The factor of 0.5 gets cancelled by the
    squared term in the derivative, so it's common to see it in the loss function.
    """
    return 0.5 * np.mean((np.array(y_hat) - np.array(y))**2)

## Defining a network

A typical neural network consist of three or more *layers*: an input layer, one or more _hidden_ layers, and an output layer.

Let's implement a network with one hidden layer. The layers are as follows:

$$ \text{Input layer:}\ \ \mathbf{x}^{(i)} $$

$$ \text{Hidden layer:}\ \ \mathbf{a}_1^{(i)} = \sigma ( \mathbf{W}_1 \mathbf{x}^{(i)} + \mathbf{b}_1) $$

$$ \text{Output layer:}\ \ \hat{\mathbf{y}}^{(i)} = \mathbf{W}_2 \mathbf{a}_1^{(i)} + \mathbf{b}_2 $$

where $\mathbf{x}^{(i)}$ is the $i$-th sample of the input data $\mathbf{X}$. $\mathbf{W}_1, \mathbf{b}_1, \mathbf{W}_2, \mathbf{b}_2$ are the weight matrices and bias vectors for layers 1 and 2 respectively, and $\sigma$ is our nonlinear function. Applying the nonlinearity to $\mathbf{W}_1 \mathbf{x}^{(i)} + \mathbf{b}_1$ in layer 1 results in the _activation_ $\mathbf{a}_1$. The output layer yields $\hat{\mathbf{y}}^{(i)}$, the $i$-th estimate of the desired output. We're not going to apply the nonlinearity to the output, but people often do. The weights are randomly initialized and the biases start at zero; during training they will be iteratively updated to encourage the network to converge on an optimal approximation to the expected output.


Note that these are vector operations. In `Numpy` we can easily deal with this because the library understands proper matrix operations. For example, matrix multiplication is done through the `@` operator.

A forward pass of the data through the network looks like this:

In [ ]:
def forward(xi, W1, b1, W2, b2, activation):
    z1 = W1 @ xi + b1
    a1 = activation(z1)
    z2 = W2 @ a1 + b2  # n.b. z2 is y_hat
    return z2, a1

Below is a picture of a neural network similar to the one we're building:

![image](../images/figure_1.png)

## How does a neural net learn?

The short version is that we show the system a bunch of corresponding input/output pairs we want it to learn, and we show it these pairs thousands of times. Every time we do so, we move the **W**'s and **b**'s in whatever direction makes the outputs of the network more similar to the known output we're trying to teach it. 

    For each training example:
        For each layer:
          - Calculate the error.
          - Calculate weight gradient.
          - Update weights.
          - Calculate the bias gradient.
          - Update biases.

What's all this about gradients?

In order to learn, the network will have to find the parameters (weights and biases) that result in the smallest loss. We'll use gradient descent for this. 

<img src="../images/gradient_descent.png" width="800px" />

This is straightforward for the output layer. That's why we needed the derivative in the activation functions, and we need to know the derivative for the `loss()` function.

The error on the output layer for a given instance (data record) looks like this:

$$ E = \frac{1}{2} \left[ \hat{y}^{(i)} - y^{(i)} \right]^2 $$

where

$$ \hat{y}^{(i)} = \mathbf{w}_2 \mathbf{a}_1^{(i)} + b_2 $$

The derivative (gradient, or slope) of this function, with respect to the weight **w**<sub>2</sub>, is:

$$ \frac{\mathrm{d}E}{\mathrm{d}\mathbf{w_2}} =  \frac{\mathrm{d}E}{\mathrm{d}\hat{y}}\frac{\mathrm{d}\hat{y}}{\mathrm{d}\mathbf{w_2}} = (\hat{y} - y) \ \mathbf{a}_1$$

To calculate the gradient at the hidden layer, we need to compute the gradient of the error with respect to the weights and biases of the hidden layer.

Let's implement this as a Python function:

In [ ]:
def backward(xi, yi,
             a1, z2,
             params,
             learning_rate,
             activation
            ):

    err_output = z2 - yi  # Derivative of loss function
    grad_W2 = err_output * a1
    params['W2'] -= learning_rate * grad_W2

    grad_b2 = err_output
    params['b2'] -= learning_rate * grad_b2

    derivative = activation(a1, derivative=True)
    err_hidden = err_output * derivative * params['W2']
    grad_W1 = err_hidden[:, None] @ xi[None, :]
    params['W1'] -= learning_rate * grad_W1
    
    grad_b1 = err_hidden
    params['b1'] -= learning_rate * grad_b1
    
    return params

The trick with the `None` indexing is the same as reshaping the array. We have to do this to produce a 2D array for the `W1` gradients.

To demonstrate this backpropagation workflow, and thus that our system can learn, let's try to get the above neural network to learn the relationship between a DT log and some other logs. We're going to need some data.

## Get some data

In [ ]:
import welly

w = welly.Well.from_las('../data/R-90.las', index='original')

data = w.data_as_matrix(keys=['GR', 'NPHISS', 'RHOB', 'DT'], start=1000, stop=3500, step=0.2)

In [ ]:
data[:10]

In [ ]:
from sklearn.preprocessing import StandardScaler

X_val = data[6500:6750, :3].reshape(-1, 3)
X_train = data[6750:7750, :3].reshape(-1, 3)

scaler = StandardScaler().fit(X_train)

X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)

In [ ]:
X_train.shape, X_val.shape

In [ ]:
fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=(15, 5))

ax0.plot(X_train)
ax1.plot(X_val)

In [ ]:
import seaborn as sns

sns.displot(X_train)

In many situations, we do not need to scale the target variable. But when using gradient descent for optimization — essentially in all neural nets — we might need to worry about it. 

Very large errors may lead to exploding gradients in training and/or result in floating point overflows — especially if you're using GPUs, which use single-precision floats.

In [ ]:
y_val_ = data[6500:6750, -1]    # Keep the unscaled data.
y_train_ = data[6750:7750, -1]

target_scaler = StandardScaler().fit(y_train_.reshape(-1, 1))

y_train = target_scaler.transform(y_train_.reshape(-1, 1))
y_val = target_scaler.transform(y_val_.reshape(-1, 1))

In [ ]:
X_train.shape, y_train.shape

In [ ]:
fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=(15, 5))

ax0.plot(y_train)
ax1.plot(y_val)

## Initialize network parameters

Now we can initialize the weights and biases for our network. A common approach is to initialize the weights with small random numbers (with NumPy's `randn()` function) and the biases with zeros.

### EXERCISE

Finish the `initialize_params()` function:

In [ ]:
def initialize_params(features, units, seed=42):
    np.random.seed(seed)
    params = {
        "W1": np.random.randn(units, features),
        "b1": np.zeros(shape=units),

        # YOUR CODE HERE
        # Initialize W2 (shape is just `units`) and b2 (shape is `1`)
        
        
        # ===============
    }
    return params

In [ ]:
def initialize_params(features, units, seed=42):
    np.random.seed(seed)
    params = {
        "W1": np.random.randn(units, features),
        "b1": np.zeros(shape=units),

        "W2": np.random.randn(units),
        "b2": np.zeros(shape=1)
    }
    return params

In [ ]:
features = X_train.shape[-1]
units = 5  # Units in hidden layer.

params = initialize_params(features, units)

In [ ]:
params

Now we have a network! It just doesn't know anything.

## Prediction

To apply this (untrained) network to some data, we're going to need a `predict` function, to make inferences from the trained network. This mode of application is called **inference**.

### EXERCISE

Finish the `predict()` function.

In [ ]:
def predict(X, forward, params, activation):
    """
    Make a prediction for a given 2D input ``X``,
    using function ``forward``.
    """
    y_hats = []
    for xi in X:
        # YOUR CODE HERE
        # You need to call `forward` to set a value for `y_hat`.
        
        # ==============
        y_hats.append(y_hat.item())
    return np.array(y_hats)

In [ ]:
def predict(X, forward, params, activation):
    """
    Make a prediction for a given 2D input ``X``,
    using function ``forward``.
    """
    y_hats = []
    for xi in X:
        y_hat, _ = forward(xi, **params, activation=activation)
        y_hats.append(y_hat.item())
    return np.array(y_hats)

Let's make a prediction for our untrained network — it should be essentially random:

In [ ]:
y_pred = predict(X_train, forward, params, activation=relu)

In [ ]:
plt.plot(y_train[:200])
plt.plot(y_pred[:200])

## Training

During training, we expose the network to the input/output pairs one at a time. These pairs are called `xi` and `yi` respectively in the code. According to our diagram above, the input goes into the green slots and we adjust the orange neurons to make the red slot output from the network a tiny bit closer to the true DT result.

We do this many times. Every time we do, we calculate the mean squared error between the network's prediction and the ground-truth output. After many iterations, or *epochs*, we draw a plot which shows the total error, or loss, at each step. If the network is learning anything, we expect the loss to decrease, as the predictions are getting closer to the ground truth.

In [ ]:
# Hyperparameters.
num_epochs = 50
learning_rate = 0.001
activation = elu

# Intitialize.
data = list(zip(X_train, y_train, y_train_))
params = initialize_params(features, units)
loss_history = []

for i in tqdm(range(num_epochs)):

    # Shuffle and prepare.
    np.random.shuffle(data)
    y_, y_hat = [], []
    
    for xi, yi, y_raw in data:
        
        # Optionally do a pass for validation (omitted here).
        
        # Forward pass.
        z2, a1 = forward(xi, **params, activation=activation)

        # Back propagation.
        params = backward(xi, yi,
                          a1, z2.item(),
                          params,
                          learning_rate,
                          activation=activation
                         )
        
        # Capture actual prediction at correct scale.
        y_.append(y_raw)
        y_hat.append(target_scaler.inverse_transform(z2))

    # Compute training loss for this epoch.
    loss_history.append(loss(y_, y_hat))

The parameters of the model are now no longer random.

In [ ]:
params

They do look kind of random though. It's usually hard to 'see' what neural networks have learned. Let's look at the W1 weights only:

In [ ]:
W1 = params['W1']

plt.figure(figsize=(5, 3))
_ = plt.imshow(W1.T, aspect='auto', vmin=0.1)

If the network learned anything **useful** then the loss should have decreased during training. The loss is our measure of whatever it is we care about.

In [ ]:
fig, ax = plt.subplots(figsize=(10,3))

ax.semilogy(loss_history, label='Training loss')

ax.set_title('Mean squared error vs epoch number', fontsize=16)
ax.tick_params(axis='both', which='major', labelsize=14)
ax.grid()

plt.tight_layout()
plt.show()

In [ ]:
y_pred = predict(X_val, forward, params, activation)

The loss decreased dramatically over the course of relatively few epochs, so presumably the network has learned something. To test this theory, let's plot the outputs after training (orange) and compare them to the expected result (blue):

In [ ]:
plt.figure(figsize=(15, 3))
plt.plot(y_val)
plt.plot(y_pred)
plt.grid(c='k', alpha=0.2)

## Compare using RMS error

It's fine for the network to learn using MSE, but it's easier for humans to understand RMS error, because it has the same units as the target.

### EXERCISE

Implement an equation for the RMS error.

$$ E_\mathrm{RMS} = \sqrt{ \frac{1}{N} \sum_{i=0}^{N} (\hat{y} - y)^2 } $$

In [ ]:
def rmse(y_true, y_pred):

    # YOUR CODE HERE
    
    return rmse

In [ ]:
def rmse(y_true, y_pred):
    """
    Compute the square root of the mean squared error.
    """
    mse = np.sum((y_pred - y_true)**2) / y_true.size
    return np.sqrt(mse)

In [ ]:
rmse(y_val_, target_scaler.inverse_transform(y_pred))

## Exercise: how does this network look in `scikit-learn`?

Replicate this neural network with `sklearn.neural_network.MLPRegressor`.

You will have to read the documentation carefully. In particular, pay attention to `solver`, `activation`, `max_iter`, and `batch_size`.

Get started with this:

In [ ]:
from sklearn.neural_network import MLPRegressor

mlp = MLPRegressor(hidden_layer_sizes=(5,),
                   tol=1e-12,   # Turn off early stopping.
                   momentum=0,  # Turn off momentum                   
                   
                   # YOUR CODE HERE
                   
                  )

mlp.fit(X_train, y_train)

y_pred_skl = mlp.predict(X_val)

In [ ]:
plt.figure(figsize=(15, 3))
plt.plot(y_val_)
plt.plot(target_scaler.inverse_transform(y_pred))
plt.plot(target_scaler.inverse_transform(y_pred_skl))
plt.grid(c='k', alpha=0.2)

In [ ]:
from sklearn.neural_network import MLPRegressor

mlp = MLPRegressor(hidden_layer_sizes=(5,),
                   activation='relu',
                   alpha=0,       # No regularization.
                   solver='sgd',  # Default is 'adam'.
                   learning_rate_init=0.001,   # Default.
                   learning_rate='constant',   # Default.
                   batch_size=1,  # Default is min(200, data length)
                   max_iter=100,
                   momentum=0,    # Turn off momentum
                  )

mlp.fit(X_train, y_train)

y_pred_skl = mlp.predict(X_val)

In [ ]:
plt.figure(figsize=(15, 3))
plt.plot(y_val_)
plt.plot(target_scaler.inverse_transform(y_pred))
plt.plot(target_scaler.inverse_transform(y_pred_skl))
plt.grid(c='k', alpha=0.2)

In [ ]:
print("Scratch NN")
print(rmse(y_val_, target_scaler.inverse_transform(y_pred)))
print()
print("Sklearn NN")
print(rmse(y_val_, target_scaler.inverse_transform(y_pred_skl)))

### EXERCISE

Can you change the hyperparameters to get a better result?

In [ ]:
# Copy the solution from the last example here.
# Then change some of the parameters and see how it affects the result.

In [ ]:
from sklearn.neural_network import MLPRegressor

mlp = MLPRegressor(hidden_layer_sizes=(20,),
                   activation='relu',
                   alpha=0.001,
                   solver='adam',
                   learning_rate_init=0.001,
                   learning_rate='constant',
                   batch_size=200,
                   max_iter=1000,
                   momentum=0.9,
                  )

mlp.fit(X_train, y_train)

y_pred_skl = mlp.predict(X_val)

In [ ]:
plt.figure(figsize=(15, 3))
plt.plot(y_val_)
plt.plot(target_scaler.inverse_transform(y_pred))
plt.plot(target_scaler.inverse_transform(y_pred_skl))
plt.grid(c='k', alpha=0.2)

In [ ]:
print("Scratch NN")
print(rmse(y_val_, target_scaler.inverse_transform(y_pred)))
print()
print("Sklearn NN")
print(rmse(y_val_, target_scaler.inverse_transform(y_pred_skl)))

## Compare with PyTorch

In [ ]:
import torch
from torch import nn

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
X_train_pt = torch.tensor(X_train, dtype=torch.float32).to(device) 
y_train_pt = torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32).to(device)

traindata = torch.utils.data.TensorDataset(X_train_pt, y_train_pt)
trainloader = torch.utils.data.DataLoader(traindata)

There's a high-level approach:

In [ ]:
net = nn.Sequential(
    nn.Linear(3, 5),
    nn.ELU(), 
    nn.Linear(5, 1),
).to(device)

And a low-level approach that gives you fine-tuned control:

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(3, 5)  # aka "Fully-connected"
        self.output = nn.Linear(5, 1)

        # Optional.
        nn.init.xavier_uniform_(self.hidden.weight)
        nn.init.zeros_(self.hidden.bias)
        nn.init.xavier_uniform_(self.output.weight)
        nn.init.zeros_(self.output.bias)

    def forward(self, x):
        z1 = self.hidden(x)
        a1 = torch.nn.functional.elu(z1)
        z2 = self.output(a1)
        return z2
    
net = Net().to(device)

Training the network:

In [ ]:
lr = 0.005
weight_decay = 0.0  # L2 regularization
optimizer = torch.optim.SGD(net.parameters(), lr=lr, weight_decay=weight_decay)

criterion = nn.MSELoss()

net.train()

epochs = 100
for epoch in range(epochs):
    epoch_loss = 0.0
    for i, data in enumerate(trainloader):
        xi, yi = data
        optimizer.zero_grad()
        y_hat = net(xi)
        loss = criterion(y_hat, yi)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"# {epoch+1}  Loss {epoch_loss}")
print('Finished training')

### Evaluate the model

In [ ]:
X_val_pt = torch.tensor(X_val, dtype=torch.float).to(device)
y_val_pt = torch.tensor(y_val.reshape(-1, 1), dtype=torch.float).to(device)

valdata = torch.utils.data.TensorDataset(X_val_pt, y_val_pt)
valloader = torch.utils.data.DataLoader(valdata)

In [ ]:
net.eval()

with torch.no_grad():
    y_pred_torch = [float(net(xi)) for xi, yi in valloader]

In [ ]:
plt.figure(figsize=(15, 3))
plt.plot(y_val_)
plt.plot(target_scaler.inverse_transform(y_pred))
plt.plot(target_scaler.inverse_transform(y_pred_skl))
plt.plot(target_scaler.inverse_transform(y_pred_torch))
plt.grid(c='k', alpha=0.2)

In [ ]:
print("Scratch NN")
print(rmse(y_val_, target_scaler.inverse_transform(y_pred)))
print()
print("Sklearn NN")
print(rmse(y_val_, target_scaler.inverse_transform(y_pred_skl)))
print()
print("PyTorch NN")
print(rmse(y_val_, target_scaler.inverse_transform(y_pred_torch)))

### Saving a PyTorch model

It is possible to save the mode with `torch.save(model, PATH)`, but this is not recommended because it depends on the exact structure of the project (files, directories, etc). Instead, PyTorch docs recommend saving the model class 

We can save the model's parameters to disk:

In [ ]:
fname = "dt_model.pth"
torch.save(net.state_dict(), fname)

...and read them into a new model:

In [ ]:
saved_net = Net()
saved_net.load_state_dict(torch.load(fname))

net.eval()

with torch.no_grad():
    y_pred_torch_ = [float(saved_net(xi)) for xi, yi in valloader]

# Check it's the same as before.
np.all(y_pred_torch == y_pred_torch_)

## Compare with linear regression

### EXERCISE

Make a prediction using `sklearn.linear_model.Ridge`. How does it compare to the neural networks?

In [ ]:
from sklearn.linear_model import Ridge

# YOUR CODE HERE

# End with...
y_pred_linreg = 

In [ ]:
from sklearn.linear_model import Ridge

regr = Ridge()

regr.fit(X_train, y_train_)  # Don't need to scale y.

y_pred_linreg = regr.predict(X_val)

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(y_val_)
plt.plot(target_scaler.inverse_transform(y_pred))
plt.plot(target_scaler.inverse_transform(y_pred_skl))
plt.plot(target_scaler.inverse_transform(y_pred_torch))
plt.plot(y_pred_linreg)
plt.grid(c='k', alpha=0.2)

In [ ]:
print("Scratch NN")
print(rmse(y_val_, target_scaler.inverse_transform(y_pred)))
print()
print("Sklearn NN")
print(rmse(y_val_, target_scaler.inverse_transform(y_pred_skl)))
print()
print("PyTorch NN")
print(rmse(y_val_, target_scaler.inverse_transform(y_pred_torch)))
print()
print("Linear regression")
print(rmse(y_val_, y_pred_linreg))

---

## Optional exercises

Try to do these exercises on the NumPy implementation. But if that proves too difficult, use the `sklearn` implementation.

- Try changing the model parameters, for example using fewer units in the hidden layer. Does this help?
- Add another layer to the model. Does this help?
- Try using other activation functions than the logistic function we're currently using.
- Implement batches, RMSprop, and momentum.

### Stretch

If you've taken the Mastery class, or know about object oriented programming, write a Python `class` to hold the NumPy implementation. Copy the `keras`/`sklearn` interface as closely as possible. Related: [this awesome video from Joel Grus](https://www.youtube.com/watch?v=o64FV-ez6Gw).

## Other types of neural networks

![image](../images/nn_arch_cheatsheet.png)

---

&copy; 2021 Agile Scientific and Graham Ganssle — Content is CC-BY-SA